# 1주차 평가

# Evn

In [2]:
# imports
import argparse
import os
import random
import shutil
import json
import zipfile
import math
import copy
import collections
import re

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sentencepiece as spm
import tensorflow as tf
import tensorflow.keras.backend as K

from tqdm.notebook import tqdm, trange

# 1. 아래 두개의 logits을 softmax를 이용한 확률분포를 numpy를 이용해 직접 계산하세요.

In [ ]:
# logits
logits_p = np.random.randn(1, 10)
logits_q = np.random.randn(1, 10)
logits_p, logits_q

# 2. 위 1번 문제에서 구한 두 확률분포의 CrossEntropy를 numpy를 이용해 직접 계산하세요.

# 3. sentencepiece를 이용해 아래 내용에 따라서 변환해 보세요.

In [ ]:
text = '이날이야말로 동소문 안에서 인력거꾼 노릇을 하는 김 첨지에게는 오래간만에도 닥친 운수 좋은 날이었다.'

3.1. 위 문장을 subword 단위로 tokenize 하세요.

3.2. 위 문장을 숫자 단위로 tokenize 하세요.

3.3. 문제 3.1의 결과인 subword를 숫자로 변환하세요.

3.4. 문제 3.2.의 결과인 숫자를 subword로 변경하세요.

3.5. 문제 3.1의 결과인 subword를 원래문장으로 복원하세요.

3.6. 문제 3.2의 결과인 숫자를 원래문장으로 복원하세요.

# 4. 아래의 길이가 다른 문장들을 sentencepiece를 이용해 숫자로 변경하고 PAD(0)를 추가해서 길이가 같게 만든 후 하나의 numpy array를 만드세요.

In [ ]:
datas = {
    'text': [
        '새침하게 흐린 품이 눈이 올 듯하더니 눈은 아니 오고 얼다가 만 비가 추적추적 내리었다.',
        '이날이야말로 동소문 안에서 인력거꾼 노릇을 하는 김 첨지에게는 오래간만에도 닥친 운수 좋은 날이었다.'
        '문안에(거기도 문밖은 아니지만) 들어간답시는 앞집 마나님을 전찻길까지 모셔다 드린 것을 비롯으로 행여나 손님이 있을까 하고 정류장에서 어정어정하며 내리는 사람 하나하나에게 거의 비는 듯한 눈결을 보내고 있다가 마침내 교원인 듯한 양복장이를 동광학교(東光學校)까지 태워다 주기로 되었다.',
        '첫번에 삼십 전, 둘째 번에 오십 전 - 아침 댓바람에 그리 흔치 않은 일이었다.',
        '그야말로 재수가 옴붙어서 근 열흘 동안 돈 구경도 못한 김 첨지는 십 전짜리 백통화 서 푼, 또는 다섯 푼이 찰깍하고 손바닥에 떨어질 제 거의 눈물을 흘릴 만큼 기뻤었다.',
        '더구나 이날 이때에 이 팔십 전이라는 돈이 그에게 얼마나 유용한지 몰랐다.',
        '컬컬한 목에 모주 한 잔도 적실 수 있거니와 그보다도 앓는 아내에게 설렁탕 한 그릇도 사다줄 수 있음이다.',
        '그의 아내가 기침으로 쿨럭거리기는 벌써 달포가 넘었다.',
        '조밥도 굶기를 먹다시피 하는 형편이니 물론 약 한 첩 써 본 일이 없다.',
        '구태여 쓰려면 못 쓸 바도 아니로되 그는 병이란 놈에게 약을 주어 보내면 재미를 붙여서 자꾸 온다는 자기의 신조(信條)에 어디까지 충실하였다.',
        '따라서 의사에게 보인 적이 없으니 무슨 병인지는 알 수 없으되 반듯이 누워 가지고, 일어나기는 새로 모로도 못 눕는걸 보면 중증은 중증인 듯.',
        '병이 이대도록 심해지기는 열흘 전에 조밥을 먹고 체한 때문이다.'
    ]
}
df = pd.DataFrame(datas)
df

# 5. 아래 30개의 숫자를 numpy를 이용해 10차원 one-hot encoding으로 변경하세요.

In [ ]:
ids = np.random.randint(0, 9, (30,))
ids

# 6. 아래 문장을 word2vec skip-gram 학습이 가능하도록 inputs와 labels를 만드세요. (window size: 2, tokenizer: sentencepiece)

In [ ]:
texts = [
    '새침하게 흐린 품이 눈이 올 듯하더니 눈은 아니 오고 얼다가 만 비가 추적추적 내리었다.',
    '이날이야말로 동소문 안에서 인력거꾼 노릇을 하는 김 첨지에게는 오래간만에도 닥친 운수 좋은 날이었다.'
    '문안에(거기도 문밖은 아니지만) 들어간답시는 앞집 마나님을 전찻길까지 모셔다 드린 것을 비롯으로 행여나 손님이 있을까 하고 정류장에서 어정어정하며 내리는 사람 하나하나에게 거의 비는 듯한 눈결을 보내고 있다가 마침내 교원인 듯한 양복장이를 동광학교(東光學校)까지 태워다 주기로 되었다.',
    '첫번에 삼십 전, 둘째 번에 오십 전 - 아침 댓바람에 그리 흔치 않은 일이었다.',
    '그야말로 재수가 옴붙어서 근 열흘 동안 돈 구경도 못한 김 첨지는 십 전짜리 백통화 서 푼, 또는 다섯 푼이 찰깍하고 손바닥에 떨어질 제 거의 눈물을 흘릴 만큼 기뻤었다.',
    '더구나 이날 이때에 이 팔십 전이라는 돈이 그에게 얼마나 유용한지 몰랐다.',
    '컬컬한 목에 모주 한 잔도 적실 수 있거니와 그보다도 앓는 아내에게 설렁탕 한 그릇도 사다줄 수 있음이다.',
    '그의 아내가 기침으로 쿨럭거리기는 벌써 달포가 넘었다.',
    '조밥도 굶기를 먹다시피 하는 형편이니 물론 약 한 첩 써 본 일이 없다.',
    '구태여 쓰려면 못 쓸 바도 아니로되 그는 병이란 놈에게 약을 주어 보내면 재미를 붙여서 자꾸 온다는 자기의 신조(信條)에 어디까지 충실하였다.',
    '따라서 의사에게 보인 적이 없으니 무슨 병인지는 알 수 없으되 반듯이 누워 가지고, 일어나기는 새로 모로도 못 눕는걸 보면 중증은 중증인 듯.',
    '병이 이대도록 심해지기는 열흘 전에 조밥을 먹고 체한 때문이다.'
]

# 7. 아래 데이터를 이용해 두 문장의 유사도를 0..1 사이의 실수로 예측하는 모델을 학습하기 위한 inputs_1, inputs_2와 labels를 생성하세요.

In [ ]:
datas = {
    'score': [3.80, 3.80, 2.60, 4.25, 0.00],
    'sentence1': [
        '한 남자가 큰 플루트를 연주하고 있다.',
        '한 남자가 피자에 치즈를 뿌려놓고 있다.',
        '세 남자가 체스를 하고 있다.',
        '한 남자가 첼로를 연주하고 있다.',
        '폭풍우 클로다흐가 영국을 강타하면서 심한 강풍'
    ],
    'sentence2': [
        '남자가 플루트를 연주하고 있다.',
        '한 남자가 구운 피자에 치즈 조각을 뿌려놓고 있다.',
        '두 남자가 체스를 하고 있다.',
        '자리에 앉은 남자가 첼로를 연주하고 있다.',
        '메르켈은 나토와 라트비아의 연대를 약속한다.'
    ]
}
df = pd.DataFrame(datas)
df

# 8. 아래 데이터를 이용해 잡답 대화 모델인 Encoder-Decoder을 학습하기 위한 enc_inputs, dec_inputs와 dec_labels를 생성하세요.

In [ ]:
datas = {
    'Q': [
        '12시 땡',
        '1지망 학교 떨어졌어',
        '3박4일 놀러가고 싶다',
        '3박4일 정도 놀러가고 싶다',
        'PPL 심하네'
    ],
    'A': [
        '하루가 또 가네요.',
        '위로해 드립니다.',
        '여행은 언제나 좋죠.',
        '여행은 언제나 좋죠.',
        '눈살이 찌푸려지죠.'
    ]
}
df = pd.DataFrame(datas)
df